In [13]:
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 데이터셋 디렉토리 설정
dataset_dir = 'dataset'

# 파일 리스트
files = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.endswith('.npy')]

# 데이터 로드 및 라벨 분류
x_data = []
labels = []
for file in files:
    data = np.load(file)
    if 'next' in file:
        label = 0
    elif 'prev' in file:
        label = 1
    else:
        continue  # idle 데이터는 무시
    print(data.shape)
    for frame_seq in data:
        x_data.append(frame_seq)  # 여기서 변경하지 않음
        labels.append(label)

x_data = np.array(x_data)
labels = np.array(labels)

# 데이터 형태 변환
y_data = to_categorical(labels, num_classes=2)  # 라벨 수를 2로 변경

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=42)


(400, 20, 100)
(378, 20, 100)


In [14]:
print(x_data.shape)

(778, 20, 100)


In [15]:
# 데이터 타입 확인
print("Data type before conversion:", x_train.dtype)

# 데이터 타입을 float32로 변환
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')

# 데이터 형태 확인
print("Shape of x_train:", x_train.shape)

Data type before conversion: float64
Shape of x_train: (700, 20, 100)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# 모델 구성
seq_length = 30
model = Sequential([
    LSTM(64, activation='relu', input_shape=(seq_length, x_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')  # 출력 레이어의 뉴런 수를 2로 변경
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 콜백 정의s
callbacks = [
    ModelCheckpoint('models/model_fr20.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto'),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=50, verbose=1, mode='auto')
]

# 모델 학습
history = model.fit(
    x_train, y_train, validation_data=(x_val, y_val),
    epochs=200, callbacks=callbacks
)


Epoch 1/200
14/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5700 - loss: 1.5043
Epoch 1: val_accuracy improved from -inf to 0.87179, saving model to models/model_fr20.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5945 - loss: 1.3200 - val_accuracy: 0.8718 - val_loss: 0.3163 - learning_rate: 0.0010
Epoch 2/200
13/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9121 - loss: 0.2583 
Epoch 2: val_accuracy improved from 0.87179 to 0.96154, saving model to models/model_fr20.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9093 - loss: 0.2559 - val_accuracy: 0.9615 - val_loss: 0.0940 - learning_rate: 0.0010
Epoch 3/200
14/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9706 - loss: 0.0941 
Epoch 3: val_accuracy improved from 0.96154 to 1.00000, saving model to models/model_fr20.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9781 - loss: 0.0770 - val_accuracy: 1.0000 - val_loss: 0.0126 - learning_rate: 0.0010
Epoch 4/200
12/22 ━━━━━━━━━━━━━━━━━━━━ 0

KeyboardInterrupt: 